In [1]:
import datetime
import json
import logging
from datetime import timedelta

import alaska_utils
import dask
import eqt_utils
import matplotlib.pyplot as plt
import numpy as np
import obspy
import pandas as pd
import postprocess
import requests
import seisbench
import seisbench.models as sbm

# Progress bar for dask operations:
from dask.diagnostics import ProgressBar
from obspy.clients.fdsn import Client

pbar = dask.diagnostics.ProgressBar()
pbar.register()

import warnings

warnings.filterwarnings("ignore")

### This notebook allows you to apply EQTransformer (by way of Seisbench) to continuous seismic data from the AACSE network, and retrieve picks for the specified time period. The waveforms can also be filtered or denoised by DeepDenoiser (by way of Seisbench).

#### Choose window length (s), window overlap (s), and start and end times

In [2]:
waveform_length = 60
waveform_overlap = 0
starttime = datetime.datetime(2019, 5, 27, 10, 30)
endtime = datetime.datetime(2019, 5, 27, 11, 30)

#### Load list of stations to run the workflow on, specify by network and maximum station elevation if desired

In [10]:
# Pre=saved station list in pandas dataframe format
dfS = pd.read_parquet(
    "https://github.com/zoekrauss/alaska_catalog/raw/main/data_acquisition/alaska_stations.parquet"
)
dfS = alaska_utils.station_list(dfS,starttime,endtime,elevation=False,network=False)

#### Specify whether to apply any filtering to the waveforms before phase picking

This choice is made with the variable filt_type.

filt_type = 0 for raw waveforms\
filt_type = 1 for bandpass filtered waveforms\
filt_type = 2 to apply DeepDenoiser to the waveforms

NOTE: if specifying bandpass filter, must also specify f1 and f2 for bandpass limits

In [11]:
filt_type = 2
f1 = False
f2 = False

### Run the workflow, and output:
#### (1) pandas dataframe of all pick metadata
#### (2) list of pick dictionaries formatted for use with GAMMA

In [12]:
%%time

pick_info,gamma_picks = eqt_utils.ml_pick(
    dfS, starttime, endtime, waveform_length, waveform_overlap, filt_type, f1=f1, f2=f2
)

[########################################] | 100% Completed | 46.3s
Status: denoising data using DeepDenoiser
Status: applying EQTransformer


2022-02-23 10:31:07,631 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-02-23 10:31:16,180 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-02-23 10:31:24,958 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-02-23 10:31:32,936 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-02-23 10:31:33,587 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-02-23 10:31:48,936 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-02-23 10:31:56,922 | seisbench | WA

CPU times: user 8h 11min 38s, sys: 13min 20s, total: 8h 24min 59s
Wall time: 12min 24s


#### Below is an example of the output pick dataframe. You can see that both the raw and denoised SNR of the pick is saved for all channels, as well as the raw and denoised amplitude of the signal (for use in magnitude estimation with GAMMA), and the probability of the pick from EQTransformer.

In [66]:
pick_info.head()

,id,network,station,channel,phase,timestamp,prob,raw_amp,den_amp,z_raw_snr,z_den_snr,n_raw_snr,n_den_snr,e_raw_snr,e_den_snr
0,AV.KCG..EH,AV,KCG,EH,P,2019-05-27T09:30:19.930,0.785364,626.0,18.518290,1.371465,30.608364,2.771061,2.953440,2.190365,12.282360
1,XO.EP21..HH,XO,EP21,HH,P,2019-05-27T09:30:58.029,0.344443,2022.0,49.235231,1.437014,29.378490,1.156233,4.545892,1.762221,121.202066
2,XO.KS11..HH,XO,KS11,HH,P,2019-05-27T09:30:19.960,0.795379,6420.0,234.239622,1.702192,12.297103,1.441493,17.106580,1.482461,21.625283
3,XO.KS11..HH,XO,KS11,HH,S,2019-05-27T09:30:29.890,0.523040,6584.0,359.900045,1.829988,3.051894,1.149442,4.339950,1.243458,1.007430
4,XO.KT07..HH,XO,KT07,HH,P,2019-05-27T09:30:23.660,0.890991,1683.0,1304.664571,5.716521,408.848358,2.089210,2.041013,2.305101,3.278548


### Save to parquet

In [14]:
pick_info.to_parquet("picks_20190527_1030.parquet", version="2.6")